In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
missing_values = ["n/a", "na", "--", " "]
train = pd.read_csv('/kaggle/input/data-storm-20/Hotel-A-train.csv', na_values = missing_values)
validation = pd.read_csv('/kaggle/input/data-storm-20/Hotel-A-validation.csv', na_values = missing_values)
test = pd.read_csv('/kaggle/input/data-storm-20/Hotel-A-test.csv', na_values = missing_values)

In [ ]:
train.head()

In [ ]:
validation.head()

In [ ]:
test.head()

In [ ]:
import seaborn as sns
sns.heatmap(train.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
sns.heatmap(validation.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
sns.heatmap(test.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
train_copy = train.copy()
validation_copy = validation.copy()
test_copy = test.copy()

In [ ]:
#convert object data to dates

import datetime
train['Expected_checkin'] = pd.to_datetime(train['Expected_checkin'],dayfirst=False, yearfirst=False, format='%m/%d/%Y')
train['Expected_checkout'] = pd.to_datetime(train['Expected_checkout'],dayfirst=True, yearfirst=True, format='%m/%d/%Y')
train['Booking_date'] = pd.to_datetime(train['Booking_date'],dayfirst=True, yearfirst=True, format='%m/%d/%Y')

validation['Expected_checkin'] = pd.to_datetime(validation['Expected_checkin'],dayfirst=False, yearfirst=False, format='%m/%d/%Y')
validation['Expected_checkout'] = pd.to_datetime(validation['Expected_checkout'],dayfirst=True, yearfirst=True, format='%m/%d/%Y')
validation['Booking_date'] = pd.to_datetime(validation['Booking_date'],dayfirst=True, yearfirst=True, format='%m/%d/%Y')

test['Expected_checkin'] = pd.to_datetime(test['Expected_checkin'],dayfirst=False, yearfirst=False, format='%m/%d/%Y')
test['Expected_checkout'] = pd.to_datetime(test['Expected_checkout'],dayfirst=True, yearfirst=True, format='%m/%d/%Y')
test['Booking_date'] = pd.to_datetime(test['Booking_date'],dayfirst=True, yearfirst=True, format='%m/%d/%Y')

In [ ]:
#Calculate number of days to stay and add a new column
numOfDays_train = []
numOfDays_valid = []
numOfDays_test = []

daysBeforeStay_train = []
daysBeforeStay_valid = []
daysBeforeStay_test = []

Raws_train=train.shape[0]
Raws_valid=validation.shape[0]
Raws_test=test.shape[0]

for i in range(Raws_train):
    k=str(abs(train.Expected_checkin[i]-train.Expected_checkout[i])) #get the difference and convert to string
    x=str(abs(train.Expected_checkin[i]-train.Booking_date[i])) #days befor stay
    Days=int(k[:1]) #num.ofDays
    daysBeforeStay=int(x[:1]) #num.ofDaysBeforeStay
    numOfDays_train.append(Days)
    daysBeforeStay_train.append(daysBeforeStay)

for i in range(Raws_valid):
    k=str(abs(validation.Expected_checkin[i]-validation.Expected_checkout[i])) #get the difference and convert to string
    x=str(abs(validation.Expected_checkin[i]-validation.Booking_date[i])) #days befor stay
    Days=int(k[:1]) #num.ofDays
    daysBeforeStay=int(x[:1]) #num.ofDaysBeforeStay
    numOfDays_valid.append(Days)
    daysBeforeStay_valid.append(daysBeforeStay)

for i in range(Raws_test):
    k=str(abs(test.Expected_checkin[i]-test.Expected_checkout[i])) #get the difference and convert to string
    x=str(abs(test.Expected_checkin[i]-test.Booking_date[i])) #days befor stay
    Days=int(k[:1]) #num.ofDays
    daysBeforeStay=int(x[:1]) #num.ofDaysBeforeStay
    numOfDays_test.append(Days)
    daysBeforeStay_test.append(daysBeforeStay)
    
train.insert(10, 'days_stay', numOfDays_train, allow_duplicates = False)
validation.insert(10, 'days_stay', numOfDays_valid, allow_duplicates = False)
test.insert(10, 'days_stay', numOfDays_test, allow_duplicates = False)

train.insert(12, 'days_before_stay', daysBeforeStay_train, allow_duplicates = False)
validation.insert(12, 'days_before_stay', daysBeforeStay_valid, allow_duplicates = False)
test.insert(12, 'days_before_stay', daysBeforeStay_test, allow_duplicates = False)

In [ ]:
train.head()

In [ ]:
validation.head()

In [ ]:
test.head()

In [ ]:
#min max normalization on 'Age','Room_Rate'

train['Age'] = (train['Age']-train['Age'].min())/(train['Age'].max()-train['Age'].min())
train['Room_Rate'] = (train['Room_Rate']-train['Room_Rate'].min())/(train['Room_Rate'].max()-train['Room_Rate'].min())

validation['Age'] = (validation['Age']-validation['Age'].min())/(validation['Age'].max()-validation['Age'].min())
validation['Room_Rate'] = (validation['Room_Rate']-validation['Room_Rate'].min())/(validation['Room_Rate'].max()-validation['Room_Rate'].min())

test['Age'] = (test['Age']-test['Age'].min())/(test['Age'].max()-test['Age'].min())
test['Room_Rate'] = (test['Room_Rate']-test['Room_Rate'].min())/(test['Room_Rate'].max()-test['Room_Rate'].min())

In [ ]:
TrainSet = train.drop(['Reservation-id','Expected_checkin','Expected_checkout','Booking_date'], axis = 1)
ValidationSet = validation.drop(['Reservation-id','Expected_checkin','Expected_checkout','Booking_date'], axis = 1)
TestSet = test.drop(['Reservation-id','Expected_checkin','Expected_checkout','Booking_date'], axis = 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer 

In [ ]:
#One Hot Encoding

columns = ['Gender','Ethnicity','Educational_Level','Income','Country_region','Hotel_Type','Meal_Type','Visted_Previously',
           'Previous_Cancellations','Deposit_type','Booking_channel','Required_Car_Parking','Use_Promotion']

for col in columns:
    one_hot = pd.get_dummies(TrainSet[col], prefix=col)
    TrainSet = TrainSet.join(one_hot)

for col in columns:
    one_hot = pd.get_dummies(ValidationSet[col], prefix=col)
    ValidationSet = ValidationSet.join(one_hot)    

for col in columns:
    one_hot = pd.get_dummies(TestSet[col], prefix=col)
    TestSet = TestSet.join(one_hot) 
    
TrainSet=TrainSet.drop(columns,axis=1)
ValidationSet=ValidationSet.drop(columns,axis=1)
TestSet=TestSet.drop(columns,axis=1)

In [ ]:
#Label Encoding

#le = LabelEncoder()

#TrainSet['Reservation_Status'] = le.fit_transform(TrainSet['Reservation_Status'])
#ValidationSet['Reservation_Status'] = le.fit_transform(ValidationSet['Reservation_Status'])

In [ ]:
# By using Mapping Structure

statusMapping = {'Check-In': '1', 'Canceled':'2','No-Show':'3'}
TrainSet['Reservation_Status']=TrainSet['Reservation_Status'].map(statusMapping)
ValidationSet['Reservation_Status']=ValidationSet['Reservation_Status'].map(statusMapping)

In [ ]:
TrainSet

In [ ]:
ValidationSet.head()

In [ ]:
TestSet.head()

In [ ]:
Y_train = TrainSet['Reservation_Status']
X_train = TrainSet.drop('Reservation_Status', axis=1)

Y_test = ValidationSet['Reservation_Status']
X_test = ValidationSet.drop('Reservation_Status', axis=1)

In [ ]:
X_train

In [ ]:
Y_train

In [ ]:
X_test

In [ ]:
Y_test

# **KNN Classifier**

In [ ]:
#kNN classifier
#Number of neighbors = 5

from sklearn.neighbors import KNeighborsClassifier

KNNclassifier = KNeighborsClassifier(n_neighbors=5)
KNNclassifier.fit(X_train, Y_train)

In [ ]:
Y_pred = KNNclassifier.predict(X_test)

In [ ]:
Y_pred

# Confusion matrix

In [ ]:
#Confusion matrix and classification report
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

confusionMatrix = confusion_matrix(Y_test, Y_pred)
print('\nConfusion Matrix: \n',confusionMatrix)
print('\nClassification Report: \n',classification_report(Y_test, Y_pred))

In [ ]:
IDs = test['Reservation-id'].values.tolist()
TestSet_pred = list(KNNclassifier.predict(TestSet))

data = {'Reservation-id': IDs,'Reservation_Status':TestSet_pred}

DF = pd.DataFrame(data)
DF.to_csv('Submission.csv', index=False) 